In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV
import data_preparation.utils.data_loader as dl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

x_2023, y_2023, X_train, X_test, y_train, y_test = dl.data_loader(
    'data_preparation/db/out/output_std.csv',
)


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

stack = StackingClassifier(
    estimators=[('dt', DecisionTreeClassifier(max_depth=7, max_features='log2', criterion='entropy')),
                ('lr', LogisticRegression(C=464, solver='sag', random_state=14)),
                ('ada', AdaBoostClassifier(n_estimators=190, learning_rate=0.2, random_state=17))],
    final_estimator=LogisticRegression(C=464, solver='sag', random_state=14),
    cv=5)

result = stack.fit(X_train, y_train)

y_pred = result.predict(X_test)


print(classification_report(y_test, y_pred))

y_pred_proba = result.predict_proba(X_test)[::, 1]
fpr_basic, tpr_basic, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc_basic = metrics.roc_auc_score(y_test, y_pred_proba)
print("AUC: ", auc_basic)